In [ ]:
!pip install mtcnn
import cv2
from mtcnn import MTCNN

# Load the image
image_path = '/content/id1.jpg'  # Replace with your image filename
image = cv2.imread(image_path)

# Initialize the MTCNN detector
detector = MTCNN()

# Detect faces and landmarks in the image
faces = detector.detect_faces(image)

# Check if any faces are detected
if len(faces) > 0:
    # Get the first detected face
    face = faces[0]

    # Get the coordinates of the left and right eyes
    left_eye = face['keypoints']['left_eye']
    right_eye = face['keypoints']['right_eye']

    # Calculate the distance between eyes in pixels
    distance_pixels = abs(left_eye[0] - right_eye[0])

    print(f"Distance between eyes: {distance_pixels} pixels")
else:
    print("No face detected in the image.")

scale_factor = 0.2  # Replace with your scale factor
distance_mm = distance_pixels * scale_factor
print(f"Distance between eyes: {distance_mm:.2f} mm")

1/1 [==============================] - 0s 121ms/step
Distance between eyes: 33 pixels
Distance between eyes: 6.60 mm


In [ ]:
import cv2
from mtcnn import MTCNN

# Load the image
image_path = '/content/id1.jpg'  # Replace with your image filename
image = cv2.imread(image_path)

# Initialize the MTCNN detector
detector = MTCNN()

# Detect faces and landmarks in the image
faces = detector.detect_faces(image)

# Check if any faces are detected
if len(faces) > 0:
    # Get the first detected face
    face = faces[0]

    # Measure the known size of the reference object in millimeters
    known_object_size_mm = 44.20  # Example: credit card width

    # Measure the actual size of the reference object in pixels
    reference_object_width_pixels = abs(face['keypoints']['right_eye'][0] - face['keypoints']['left_eye'][0])
    print(f"absolute Distance between eyes: {reference_object_width_pixels:.2f} mm")
    # Calculate the conversion factor: pixels to millimeters
    conversion_factor = known_object_size_mm / reference_object_width_pixels
    print(f"Conversion factor: {conversion_factor:.2f} mm")

    # Measure the distance between eyes in pixels
    distance_pixels = abs(face['keypoints']['left_eye'][0] - face['keypoints']['right_eye'][0])
    print(f"Distance pixel: {distance_pixels:.2f} mm")

    # Convert distance from pixels to millimeters
    distance_mm = distance_pixels * conversion_factor

    print(f"Distance between eyes: {distance_mm:} mm")
else:
    print("No face detected in the image.")

1/1 [==============================] - 0s 130ms/step
absolute Distance between eyes: 33.00 mm
Conversion factor: 1.34 mm
Distance pixel: 33.00 mm
Distance between eyes: 44.2 mm


In [ ]:
import cv2
from sklearn.linear_model import LinearRegression
import numpy as np

# Simulated calibration data (actual distances in millimeters and measured distances in pixels)
calibration_data = [
    {'actual_distance_mm': 44.20, 'measured_distance_pixels': 221},
    {'actual_distance_mm': 49.00, 'measured_distance_pixels': 245},
    {'actual_distance_mm': 30.80, 'measured_distance_pixels': 154},
    {'actual_distance_mm': 6.60, 'measured_distance_pixels': 33},
    {'actual_distance_mm': 10.40, 'measured_distance_pixels': 52},
    {'actual_distance_mm': 16.60, 'measured_distance_pixels': 83},
    # ... add more data points
]

# Extract pixel distances and actual distances from calibration data
pixel_distances = [data_point['measured_distance_pixels'] for data_point in calibration_data]
actual_distances_mm = [data_point['actual_distance_mm'] for data_point in calibration_data]

# Convert lists to numpy arrays
pixel_distances = np.array(pixel_distances).reshape(-1, 1)
actual_distances_mm = np.array(actual_distances_mm)

# Train a linear regression model to predict actual distances from pixel distances
regressor = LinearRegression()
regressor.fit(pixel_distances, actual_distances_mm)

# Test the calibration with a new image
new_pixel_distance = 250  # Measure new pixel distance between eyes in an image

# Predict the actual distance using the trained regression model
predicted_actual_distance_mm = regressor.predict([[new_pixel_distance]])[0]

print("Predicted distance between eyes:", predicted_actual_distance_mm, "mm")


Predicted distance between eyes: 49.99999999999999 mm


In [ ]:
import cv2
from sklearn.linear_model import LinearRegression
import numpy as np

# Load the image
image_path = '/content/id1.jpg'
image = cv2.imread(image_path)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Load Haar cascade classifiers for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Detect faces in the grayscale image
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# If a face is detected, proceed to detect the eyes' positions
if len(faces) > 0:
    x, y, w, h = faces[0]
    face_roi = gray_image[y:y+h, x:x+w]

    # Detect eyes within the detected face region
    eyes = eye_cascade.detectMultiScale(face_roi)

    if len(eyes) >= 2:  # Assuming at least two eyes are detected
        # Get the x-coordinates of the left and right eyes
        left_eye_x = x + eyes[0][0]
        right_eye_x = x + eyes[1][0] + eyes[1][2]

        # Simulated calibration data (actual distances in millimeters and measured distances in pixels)
        calibration_data = [
            {'actual_distance_mm': 44.20, 'measured_distance_pixels': 221},
            {'actual_distance_mm': 49.00, 'measured_distance_pixels': 245},
            # ... add more data points
        ]

        # Extract pixel distances and actual distances from calibration data
        pixel_distances = [data_point['measured_distance_pixels'] for data_point in calibration_data]
        actual_distances_mm = [data_point['actual_distance_mm'] for data_point in calibration_data]

        # Convert lists to numpy arrays
        pixel_distances = np.array(pixel_distances).reshape(-1, 1)
        actual_distances_mm = np.array(actual_distances_mm)

        # Train a linear regression model to predict actual distances from pixel distances
        regressor = LinearRegression()
        regressor.fit(pixel_distances, actual_distances_mm)

        # Calculate the new pixel distance between eyes
        new_pixel_distance = abs(right_eye_x - left_eye_x)

        # Predict the actual distance using the trained regression model
        predicted_actual_distance_mm = regressor.predict([[new_pixel_distance]])[0]

        print("Predicted distance between eyes:", predicted_actual_distance_mm, "mm")
    else:
        print("Less than two eyes detected in the face region.")
else:
    print("No face detected in the image.")

Predicted distance between eyes: 11.800000000000033 mm


In [ ]:
import cv2
import numpy as np
from sklearn.linear_model import LinearRegression

# Load the image
image_path = '/content/id1.jpg'
image = cv2.imread(image_path)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Simulated calibration data (actual distances in millimeters and measured distances in pixels)
calibration_data = [
    {'actual_distance_mm': 44.20, 'measured_distance_pixels': 221},
    {'actual_distance_mm': 49.00, 'measured_distance_pixels': 245},
    {'actual_distance_mm': 30.80, 'measured_distance_pixels': 154},
    {'actual_distance_mm': 6.60, 'measured_distance_pixels': 33},
    {'actual_distance_mm': 10.40, 'measured_distance_pixels': 52},
    {'actual_distance_mm': 16.60, 'measured_distance_pixels': 83},
]

# Extract pixel distances and actual distances from calibration data
pixel_distances = [data_point['measured_distance_pixels'] for data_point in calibration_data]
actual_distances_mm = [data_point['actual_distance_mm'] for data_point in calibration_data]

# Convert lists to numpy arrays
pixel_distances = np.array(pixel_distances).reshape(-1, 1)
actual_distances_mm = np.array(actual_distances_mm)

# Train a linear regression model to predict actual distances from pixel distances
regressor = LinearRegression()
regressor.fit(pixel_distances, actual_distances_mm)

# Load Haar cascade classifiers for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Detect faces in the grayscale image
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# If a face is detected, proceed to detect the eyes' positions
if len(faces) > 0:
    x, y, w, h = faces[0]
    face_roi = gray_image[y:y+h, x:x+w]

    # Detect eyes within the detected face region
    eyes = eye_cascade.detectMultiScale(face_roi)

    if len(eyes) >= 2:  # Assuming at least two eyes are detected
        # Get the x-coordinates of the left and right eyes
        left_eye_x = x + eyes[0][0]
        right_eye_x = x + eyes[1][0] + eyes[1][2]

        # Calculate the new pixel distance between eyes
        new_pixel_distance = abs(right_eye_x - left_eye_x)

        if new_pixel_distance > 150:  # If pixel distance is greater than 150, use it directly
            predicted_actual_distance_mm = regressor.predict([[new_pixel_distance]])[0]
            print("Predicted distance between eyes:", predicted_actual_distance_mm, "mm")
        else:  # If pixel distance is less than or equal to 150, scale the image using a reference object
            # Measure the known size of the reference object in millimeters
            known_object_size_mm = 65  # Example: reference object width

            # Measure the actual size of the reference object in pixels
            reference_object_width_pixels = w  # Use face width as the reference object width

            # Calculate the conversion factor: pixels to millimeters
            conversion_factor = known_object_size_mm / reference_object_width_pixels

            # Scale the image using the conversion factor
            scaled_image = cv2.resize(image, None, fx=conversion_factor, fy=conversion_factor)
            gray_scaled_image = cv2.cvtColor(scaled_image, cv2.COLOR_BGR2GRAY)

            # Detect faces in the scaled grayscale image
            scaled_faces = face_cascade.detectMultiScale(gray_scaled_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            if len(scaled_faces) > 0:
                # Proceed with detecting eyes and measuring the distance as before
                scaled_x, scaled_y, scaled_w, scaled_h = scaled_faces[0]
                scaled_face_roi = gray_scaled_image[scaled_y:scaled_y+scaled_h, scaled_x:scaled_x+scaled_w]

                # Detect eyes within the scaled detected face region
                scaled_eyes = eye_cascade.detectMultiScale(scaled_face_roi)

                if len(scaled_eyes) >= 2:  # Assuming at least two eyes are detected in the scaled image
                    # Get the x-coordinates of the left and right eyes in the scaled image
                    scaled_left_eye_x = scaled_x + scaled_eyes[0][0]
                    scaled_right_eye_x = scaled_x + scaled_eyes[1][0] + scaled_eyes[1][2]

                    # Calculate the new scaled pixel distance between eyes
                    new_scaled_pixel_distance = abs(scaled_right_eye_x - scaled_left_eye_x)

                    # Predict the actual distance using the trained regression model
                    predicted_actual_distance_mm = regressor.predict([[new_scaled_pixel_distance]])[0]
                    print("Predicted distance between eyes (scaled image):", predicted_actual_distance_mm, "mm")
                else:
                    print("Less than two eyes detected in the scaled face region.")
            else:
                print("No face detected in the scaled image.")

    else:
        print("Less than two eyes detected in the face region.")
else:
    print("No face detected in the image.")


Predicted distance between eyes (scaled image): 0.6 mm


In [ ]:
import cv2

def detect(im):
    eyepair_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_mcs_eyepair_small.xml")
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

    if eyepair_cascade.empty() or eye_cascade.empty():
        raise Exception("Cascade classifier loading failed.")

    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    eyepairs = eyepair_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    for (x, y, w, h) in eyepairs:
        cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2)

        roi_gray = gray[y:y+h, x:x+w]
        roi_color = im[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

    cv2.imwrite("result.png", im)

image_path = "/content/f1.jpg"  # Replace with the actual image path
image = cv2.imread(image_path)

if image is None:
    raise Exception("Image not loaded.")

detect(image)


Exception: ignored

In [ ]:
!pip install mtcnn
import cv2
from mtcnn import MTCNN
import math

# Load the image
image_path = '/content/f1.jpg'  # Replace with your image filename
image = cv2.imread(image_path)

# Initialize the MTCNN detector
detector = MTCNN()

# Known physical distance between pupils in the reference image (in mm)
known_physical_distance = 44.20  # Replace with your known distance

# Known pixel distance between pupils in the reference image
known_pixel_distance = 36.0  # Replace with the corresponding pixel distance in your reference image

# Detect faces and landmarks in the image
faces = detector.detect_faces(image)

# Check if any faces are detected
if len(faces) > 0:
    # Get the first detected face
    face = faces[0]

    # Get the coordinates of the left and right eyes (pupils)
    left_eye = face['keypoints']['left_eye']
    right_eye = face['keypoints']['right_eye']

    # Calculate the distance between pupils in pixels using Euclidean distance
    distance_pixels = math.sqrt((left_eye[0] - right_eye[0]) ** 2 + (left_eye[1] - right_eye[1]) ** 2)

    # Calculate the scale factor to convert pixels to millimeters
    scale_factor = known_physical_distance / known_pixel_distance

    # Convert distance from pixels to millimeters
    distance_mm = distance_pixels * scale_factor

    print(f"Distance between pupils of eyes: {distance_mm:.2f} mm")
else:
    print("No face detected in the image.")


1/1 [==============================] - 0s 128ms/step
Distance between pupils of eyes: 271.34 mm
